In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from typing import Optional, cast, Dict, Any

import torch

import transformers
from transformers import AutoModel, AutoConfig, AutoTokenizer

from omegaconf import DictConfig
from omegaconf import OmegaConf as om

from src.flex_bert import *
from src.evals.data import *

/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:958: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:1017: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., 

In [3]:
def consume_prefix_in_state_dict_if_present(
    state_dict, prefix
):
    r"""Strip the prefix in state_dict in place, if any.

    ..note::
        Given a `state_dict` from a DP/DDP model, a local model can load it by applying
        `consume_prefix_in_state_dict_if_present(state_dict, "module.")` before calling
        :meth:`torch.nn.Module.load_state_dict`.

    Args:
        state_dict (OrderedDict): a state-dict to be loaded to the model.
        prefix (str): prefix.
    """
    keys = sorted(state_dict.keys())
    for key in keys:
        if key.startswith(prefix):
            newkey = key[len(prefix) :]
            state_dict[newkey] = state_dict.pop(key)

    # also strip the prefix in metadata if any.
    if "_metadata" in state_dict:
        metadata = state_dict["_metadata"]
        for key in list(metadata.keys()):
            # for the metadata dict, the key can be:
            # '': for the DDP module, which we want to remove.
            # 'module': for the actual model.
            # 'module.xx.xx': for the rest.

            if len(key) == 0:
                continue
            newkey = key[len(prefix) :]
            metadata[newkey] = metadata.pop(key)

In [4]:
state_dict = torch.load("/home/public/span/MATH_DPO/modern_bert_test/bert24/checkpoints/ep1-ba542-rank0.pt")
state_dict = state_dict['state']['model']
consume_prefix_in_state_dict_if_present(state_dict, "model.")
torch.save(state_dict, "/home/public/span/MATH_DPO/modern_bert_test/bert24/checkpoints/correct_names.pt")

/home/public/span/temp/ipykernel_35250/1181012206.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/home/public/span/MATH_DPO/modern_bert_test/be

In [ ]:
with open("/home/public/span/MATH_DPO/modern_bert_test/bert24/yamls/test/sequence_classification_og.yaml") as f:
    yaml_config = om.load(f)

cfg = cast(DictConfig, yaml_config)

In [6]:
model = create_flex_bert_classification(
    num_labels=cfg.model.num_labels,
    pretrained_checkpoint=cfg.model.pretrained_checkpoint,
    model_config=cfg.model.model_config,
    tokenizer_name=cfg.tokenizer_name
)

/home/public/span/MATH_DPO/modern_bert_test/bert24/src/bert_layers/model.py:1300: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pretrained_checkpoint

In [7]:
import datasets
ds = datasets.load_dataset("sarahpann/reward_bench_processed")

In [8]:
dataloader = torch.utils.data.DataLoader(ds['train'])

In [9]:
all_dsets = {'llmbar-adver-manual', 'hep-python', 'refusals-dangerous', 
                 'mt-bench-med', 'refusals-offensive', 'alpacaeval-length', 
                 'llmbar-natural', 'alpacaeval-hard', 'mt-bench-easy', 
                 'mt-bench-hard', 'llmbar-adver-GPTOut', 'hep-rust', 
                 'hep-java', 'donotanswer', 'hep-js', 'llmbar-adver-GPTInst',
                   'math-prm', 'hep-go', 'alpacaeval-easy', 'xstest-should-respond', 
                   'llmbar-adver-neighbor', 'hep-cpp', 'xstest-should-refuse'}

categories = {
    "chat": {'alpacaeval-easy', 'alpacaeval-length', 'alpacaeval-hard', 
                'mt-bench-easy', 'mt-bench-med'},
    "chat_hard": {'mt-bench-hard', 'llmbar-natural', 'llmbar-adver-neighbor',
                    'llmbar-adver-GPTInst', 'llmbar-adver-GPTOut', 'llmbar-adver-manual'},
    "safety": {'refusals-dangerous', 'refusals-offensive', 'xstest-should-respond', 
                'xstest-should-refuse', 'xstest-should-respond', 'donotanswer'},
    "reasoning": {'math-prm', 'hep-cpp', 'hep-go', 'hep-java', 'hep-js', 'hep-python', 'hep-rust'},
}

inverse_map = {v: k for k in categories for v in categories[k]}
category_tallies = {k: 0 for k in categories.keys()}
category_amounts = {k: 0 for k in categories.keys()}

In [10]:
for ex in dataloader:
    print(ex)
    break

{'chosen': ['What are the names of some famous actors that started their careers on Broadway?\nSeveral famous actors started their careers on Broadway before making it big in film and television. Here are a few notable examples:\n\n1. Sarah Jessica Parker - Before she was Carrie Bradshaw on "Sex and the City," Sarah Jessica Parker was a Broadway star, having appeared in productions like "Annie" as a child.\n\n2. Meryl Streep - Meryl Streep\'s early career included Broadway productions such as "Trelawny of the \'Wells\'" and "A Memory of Two Mondays / 27 Wagons Full of Cotton."\n\n3. Hugh Jackman - Hugh Jackman won a Tony Award for his role in "The Boy from Oz" and has been known for his stage work as well as his film career.\n\n4. Sutton Foster - Known for her television role in "Younger," Sutton Foster is also a Broadway legend with leading roles in shows like "Thoroughly Modern Millie" and "Anything Goes."\n\n5. Kristen Bell - Before she was the voice of Anna in "Frozen" or the star 

In [11]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bclavie/olmo_bert_template")

In [16]:
model = model.model

In [20]:
for ex in dataloader:
    print(ex.keys())
    break

dict_keys(['chosen', 'rejected', 'og_dataset'])


In [ ]:
inverse_map['']

{'mt-bench-med': 'chat',
 'mt-bench-easy': 'chat',
 'alpacaeval-length': 'chat',
 'alpacaeval-hard': 'chat',
 'alpacaeval-easy': 'chat',
 'llmbar-adver-neighbor': 'chat_hard',
 'mt-bench-hard': 'chat_hard',
 'llmbar-adver-GPTOut': 'chat_hard',
 'llmbar-natural': 'chat_hard',
 'llmbar-adver-GPTInst': 'chat_hard',
 'llmbar-adver-manual': 'chat_hard',
 'refusals-dangerous': 'safety',
 'xstest-should-respond': 'safety',
 'refusals-offensive': 'safety',
 'donotanswer': 'safety',
 'xstest-should-refuse': 'safety',
 'hep-java': 'reasoning',
 'hep-rust': 'reasoning',
 'hep-cpp': 'reasoning',
 'hep-go': 'reasoning',
 'math-prm': 'reasoning',
 'hep-js': 'reasoning',
 'hep-python': 'reasoning'}

In [26]:
# model = model.model
model.to("cuda")
model.eval()

with torch.no_grad():
    for example in dataloader:
        tokenized_chosen = tokenizer(example['chosen'], return_tensors="pt")
        tokenized_rejected = tokenizer(example['rejected'], return_tensors="pt")

        chosen_out = model(tokenized_chosen['input_ids'].to("cuda"))
        rejected_out = model(tokenized_rejected['input_ids'].to("cuda"))

        chosen_pred = torch.argmax(chosen_out.logits, dim=1)
        rejected_pred = torch.argmax(rejected_out.logits, dim=1)

        if chosen_pred == 1:
            category_tallies[inverse_map[example['og_dataset'][0]]] += 1

        if rejected_pred == 0:
            category_tallies[inverse_map[example['og_dataset'][0]]] += 1

        category_amounts[inverse_map[example['og_dataset'][0]]] += 2


print(f"{category_tallies=}")

# get accuracies
accuracy_dict = {k: category_tallies[k] / category_amounts[k] for k in category_tallies.keys()}
print(f"{accuracy_dict=}")

# get total accuracy
total_correct = sum(category_tallies.values())
total_amount = sum(category_amounts.values())
total_accuracy = total_correct / total_amount
print(f"{total_accuracy=}")

category_tallies={'chat': 2646, 'chat_hard': 446, 'safety': 664, 'reasoning': 1428}
accuracy_dict={'chat': 0.5317524115755627, 'chat_hard': 0.48060344827586204, 'safety': 0.4486486486486487, 'reasoning': 0.4989517819706499}
total_accuracy=0.5059535428459887
